In [1]:
#Import Libraries and Packages

import os
import json

# keras imports
from tensorflow.keras.utils import img_to_array, load_img
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from keras.models import Model


import h5py
import numpy as np
import pickle as pk

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Loading Models for various checks
first_check = VGG16(weights='imagenet')
second_check = pk.load(open("YourPath/cdmg/classifier.pickle", 'rb')) #damaged vs whole
third_check = pk.load(open("YourPath/FRS/classifier.pickle", 'rb')) # Front Rear Side
fourth_check = pk.load(open("YourPath/Severity/classifier.pickle", 'rb')) # Minor Moderate Severe

In [4]:
# Loading  valid categories for identifying cars using VGG16
with open('cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [5]:
cat_counter

Counter({('n04252225', 'snowplow'): 7.086044100462459,
         ('n03478589', 'half_track'): 0.8668185896240175,
         ('n04461696', 'tow_truck'): 18.77435178961605,
         ('n03126707', 'crane'): 0.1037681084126234,
         ('n03417042', 'garbage_truck'): 1.4159725971985608,
         ('n02701002', 'ambulance'): 5.5628304437268525,
         ('n03977966', 'police_van'): 10.42100810923148,
         ('n03345487', 'fire_engine'): 2.669070777017623,
         ('n03670208', 'limousine'): 53.6091693774797,
         ('n03594945', 'jeep'): 30.02313027821947,
         ('n03930630', 'pickup'): 65.00294746865984,
         ('n04467665', 'trailer_truck'): 3.2392273662262596,
         ('n02814533', 'beach_wagon'): 67.8161769274011,
         ('n03770679', 'minivan'): 132.04886324098334,
         ('n02974003', 'car_wheel'): 17.176702222801396,
         ('n03100240', 'convertible'): 91.41380487324204,
         ('n03459775', 'grille'): 18.050603751893505,
         ('n04285008', 'sports_car'): 116.25

In [6]:
# shortlisting top 27 Categories that VGG16 stores for cars (Can be altered for less or more)
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [7]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n03594945', 'jeep'),
 ('n03769881', 'minibus'),
 ('n04461696', 'tow_truck'),
 ('n03459775', 'grille'),
 ('n02930766', 'cab'),
 ('n02974003', 'car_wheel'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04252225', 'snowplow'),
 ('n02701002', 'ambulance'),
 ('n04065272', 'recreational_vehicle'),
 ('n04467665', 'trailer_truck'),
 ('n03445924', 'golfcart'),
 ('n03345487', 'fire_engine'),
 ('n03791053', 'motor_scooter'),
 ('n03776460', 'mobile_home'),
 ('n04252077', 'snowmobile'),
 ('n02704792', 'amphibian'),
 ('n03417042', 'garbage_truck'),
 ('n02965783', 'car_mirror')]

In [8]:
CLASS_INDEX = None

In [9]:
CLASS_INDEX_PATH = 'YourPath/imagenet_class_index.json'

In [10]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [11]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [12]:
def car_categories_check(img_224, model):
    print ("Validating that this is a picture of your car...")
    out = model.predict(img_224)
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print ("Car Check Passed!!!")
            print ("\n")
            return True 
    return False

In [13]:
def car_damage_check(classifier):
    print ("Validating that damage exists...")
    base_model = first_check
    train_labels = ['Damaged', 'Not Damaged']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print ("Validation complete - proceeding to location and severity determination")
        print ("\n")
        return True 
    else:
        return False

In [14]:
def location_assessment(classifier):
    print ("Validating the damage area - Front, Rear or Side")
    base_model = first_check
    train_labels = ['Front Damage', 'Rear Damage', 'Side Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car is damaged at - " + train_labels[preds[0]])
    print ("Location assesment complete")
    print("\n")

In [15]:
def severity_assessment(classifier):
    print ("Validating the Severity...")
    base_model = first_check
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    
    model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    preds = classifier.predict(flat)
    prediction = train_labels[preds[0]]
    print ("Your Car damage impact is - " + train_labels[preds[0]])
    print ("Severity assesment complete")

In [16]:
def engine(img):
    while True:
        try:
            img_224 = prepare_img_224(img)
            g1 = car_categories_check(img_224, first_check)

            if g1 is False:
                print ("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            g2 = car_damage_check(second_check)

            if g2 is False:
                print ("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            x = location_assessment(third_check)
            y = severity_assessment(fourth_check)
            break
        except:
            print ("Image not accessible. Please try again.")
            break

In [21]:
img_path = 'C:/Users/Eisu/Desktop/engine/frontdamage.jpg'

In [22]:
engine(img_path)

Validating that this is a picture of your car...
1/1 [==============================] - 0s 454ms/step
Car Check Passed!!!


Validating that damage exists...
1/1 [==============================] - 0s 351ms/step
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
1/1 [==============================] - 0s 419ms/step
Your Car is damaged at - Front Damage
Location assesment complete


Validating the Severity...
1/1 [==============================] - 0s 389ms/step
Your Car damage impact is - Severe Damage
Severity assesment complete
